In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import os

## Tokenizer

In [2]:
class SimpleTokenizer:
    def __init__(self, texts, special_tokens=["<pad>", "<sos>", "<eos>", "<unk>", "NEW_LINE", "INDENT", "DEDENT"]):
        words = set()
        for line in texts:
            words.update(line.strip().split())
        self.word2idx = {t

        ok: idx for idx, tok in enumerate(special_tokens)}
        for word in sorted(words):
            if word not in self.word2idx:
                self.word2idx[word] = len(self.word2idx)
        self.idx2word = {idx: tok for tok, idx in self.word2idx.items()}
        self.pad_token_id = self.word2idx["<pad>"]
        self.sos_token_id = self.word2idx["<sos>"]
        self.eos_token_id = self.word2idx["<eos>"]
        self.unk_token_id = self.word2idx["<unk>"]

    def encode(self, text, max_len=256):
        tokens = ["<sos>"] + text.strip().split() + ["<eos>"]
        token_ids = [self.word2idx.get(tok, self.unk_token_id) for tok in tokens]
        token_ids = token_ids[:max_len]
        return token_ids + [self.pad_token_id] * (max_len - len(token_ids))

    def decode(self, ids):
        words = [self.idx2word.get(i, "<unk>") for i in ids if i not in (self.pad_token_id, self.eos_token_id, self.sos_token_id)]
        return " ".join(words).strip()

## Dataset

In [3]:
class CodeTranslationDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_len=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]
        src_ids = self.tokenizer.encode(src, self.max_len)
        tgt_ids = self.tokenizer.encode(tgt, self.max_len)
        labels = [i if i != self.tokenizer.pad_token_id else -100 for i in tgt_ids]
        return {
            "input_ids": torch.tensor(src_ids),
            "labels": torch.tensor(labels),
            "lengths": (torch.tensor(src_ids) != self.tokenizer.pad_token_id).sum()
        }


## Attention Model

In [4]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Parameter(torch.rand(hid_dim))

    def forward(self, hidden, encoder_outputs):
        src_len = encoder_outputs.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(hidden.size(0), 1).unsqueeze(1)
        attn_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attn_weights, dim=1)

class AttnSeq2Seq(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.encoder = nn.GRU(emb_dim, hid_dim, batch_first=True)
        self.decoder = nn.GRU(emb_dim + hid_dim, hid_dim, batch_first=True)
        self.attn = Attention(hid_dim)
        self.fc_out = nn.Linear(hid_dim * 2, vocab_size)
        self.pad_idx = pad_idx

    def forward(self, src, trg, lengths):
        embedded_src = self.embedding(src)
        packed = pack_padded_sequence(embedded_src, lengths.cpu(), batch_first=True, enforce_sorted=False)
        encoder_outputs, hidden = self.encoder(packed)
        encoder_outputs, _ = pad_packed_sequence(encoder_outputs, batch_first=True)

        decoder_input = trg.clone()
        decoder_input[decoder_input == -100] = self.pad_idx
        embedded_trg = self.embedding(decoder_input[:, :-1])
        outputs = []
        input_step = embedded_trg[:, 0].unsqueeze(1)

        for t in range(embedded_trg.size(1)):
            a = self.attn(hidden[-1], encoder_outputs).unsqueeze(1)
            weighted = torch.bmm(a, encoder_outputs)
            rnn_input = torch.cat((input_step, weighted), dim=2)
            output, hidden = self.decoder(rnn_input, hidden)
            prediction = self.fc_out(torch.cat((output, weighted), dim=2)).squeeze(1)
            outputs.append(prediction)
            if t + 1 < embedded_trg.size(1):
                input_step = embedded_trg[:, t + 1].unsqueeze(1)
        return torch.stack(outputs, dim=1)

## Training

In [5]:
def train(model, dataloader, optimizer, criterion, device, scaler):
    model.train()
    epoch_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        src = batch["input_ids"].to(device)
        trg = batch["labels"].to(device)
        lengths = batch["lengths"].to(device)
        optimizer.zero_grad()
        with autocast():
            output = model(src, trg, lengths)
            output = output.reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)


## Evalution

In [6]:
def evaluate_bleu(model, val_pairs, tokenizer, device, max_len=256, num_samples=5):
    model.eval()
    smoothie = SmoothingFunction().method4
    total_bleu = 0
    for i in range(min(num_samples, len(val_pairs))):
        src_text, tgt_text = val_pairs[i]
        src_ids = tokenizer.encode(src_text)
        src_tensor = torch.tensor(src_ids).unsqueeze(0).to(device)
        with torch.no_grad(), autocast():
            lengths = (src_tensor != tokenizer.pad_token_id).sum(dim=1)
            encoder_outputs, hidden = model.encoder(
                pack_padded_sequence(model.embedding(src_tensor), lengths.cpu(), batch_first=True, enforce_sorted=False)
            )
            encoder_outputs, _ = pad_packed_sequence(encoder_outputs, batch_first=True)
            token = torch.tensor([[tokenizer.sos_token_id]], device=device)
            generated = []
            for _ in range(max_len):
                embedded = model.embedding(token)
                a = model.attn(hidden[-1], encoder_outputs).unsqueeze(1)
                weighted = torch.bmm(a, encoder_outputs)
                rnn_input = torch.cat((embedded, weighted), dim=2)
                output, hidden = model.decoder(rnn_input, hidden)
                prediction = model.fc_out(torch.cat((output, weighted), dim=2)).squeeze(1)
                token = prediction.argmax(dim=1, keepdim=True)
                if token.item() == tokenizer.eos_token_id:
                    break
                generated.append(token.item())
            pred_text = tokenizer.decode(generated)
            bleu = sentence_bleu([tgt_text.split()], pred_text.split(), smoothing_function=smoothie)
            print(f"\nBLEU: {bleu:.4f}\nPred: {pred_text}\nRef : {tgt_text}")
            total_bleu += bleu
    return total_bleu / num_samples


## Helper Functions

In [7]:
def save_model(model, path="best_model.pt"):
    print("Saving model...")
    torch.save(model.state_dict(), path)

def load_model(model, path="best_model.pt"):
    model.load_state_dict(torch.load(path))
    model.eval()

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best = float('inf')
        self.stop = False

    def __call__(self, val_loss):
        if val_loss < self.best:
            self.best = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

## Training Loop

In [8]:
if __name__ == "__main__":
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def load_pairs(py_file, js_file):
        with open(py_file, encoding="utf-8") as f1, open(js_file, encoding="utf-8") as f2:
            src_lines = [l.strip() for l in f1 if l.strip()]
            tgt_lines = [l.strip() for l in f2 if l.strip()]
        return list(zip(src_lines, tgt_lines))

    train_pairs = load_pairs(
        "data/generation/pair_data_tok_full/Python-Javascript/train-Python-Javascript-tok.py",
        "data/generation/pair_data_tok_full/Python-Javascript/train-Python-Javascript-tok.js"
    )
    val_pairs = load_pairs(
        "data/generation/pair_data_tok_full/Python-Javascript/val-Python-Javascript-tok.py",
        "data/generation/pair_data_tok_full/Python-Javascript/val-Python-Javascript-tok.js"
    )

    all_texts = [src for src, _ in train_pairs + val_pairs] + [tgt for _, tgt in train_pairs + val_pairs]

    tokenizer = SimpleTokenizer(all_texts)
    VOCAB_SIZE = len(tokenizer.word2idx)

    train_ds = CodeTranslationDataset(train_pairs, tokenizer)
    val_ds   = CodeTranslationDataset(val_pairs, tokenizer)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
    val_loader   = DataLoader(val_ds, batch_size=32, num_workers=2)

    model     = AttnSeq2Seq(VOCAB_SIZE, 256, 512, pad_idx=tokenizer.pad_token_id).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    scaler    = GradScaler()
    early_stopper = EarlyStopping(patience=4)

    for epoch in range(10):
        print(f"\nEpoch {epoch+1}")
        if epoch == 1:
            model = torch.compile(model)
        train_loss = train(model, train_loader, optimizer, criterion, DEVICE, scaler)
        bleu = evaluate_bleu(model, val_pairs, tokenizer, DEVICE)
        print(f"Train Loss: {train_loss:.4f} | BLEU: {bleu:.4f}")
        if bleu > 0.5:
            save_model(model)
        early_stopper(train_loss)
        if early_stopper.stop:
            break

    save_model(model, "rnn_model.pt")

/tmp/ipykernel_109529/3844681980.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler    = GradScaler()



Epoch 1


Training:   0%|          | 0/256 [00:00<?, ?it/s]/tmp/ipykernel_109529/565682924.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 256/256 [01:47<00:00,  2.37it/s]



BLEU: 0.0092
Pred: function check ( str ) { let count = 0 ; for ( let i = 0 ; i < n ; i ++ ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 ) { if ( i == 0 )
Ref : function Conversion ( centi ) { let pixels = ( 96 * centi ) / 2.54 ; document . write ( pixels ) ; return 0 ; } let centi = 15 ; Conversion ( centi )

BLEU: 0.1924
Pred: function nthTerm ( N , arr , N , K ) { let ans = 0 ; for ( let i = 0 ; i < N ; i ++ ) { if ( arr [ i ] == arr [ i ] ) { if ( arr [ i ] == arr [ i ] ) { if ( arr [ i ] == arr [ i ] ) { if ( arr [ i ] == 

Training:   0%|          | 1/256 [00:03<13:51,  3.26s/it]/tmp/ipykernel_109529/565682924.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 256/256 [01:45<00:00,  2.43it/s]
/tmp/ipykernel_109529/3309552307.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


...


BLEU: 0.0322
Pred: function perimeter ( str1 ) { let len = ( Math . ceil ( ) ) / 100 ; document . write ( " " + " " + " " ) ; document . write ( " " + " " + " " ) ; document . write ( " " + " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ; document . write ( " " + " " ) ;
Ref : function Conversion ( centi ) { let pixels = ( 96 * centi ) / 2.54 ; document . write ( pixels ) ; return 0 ; } let centi = 15 ; Conversion ( centi )

BLEU: 0.6528
Pred: function reverse ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let i = 0 ; i < M ; i ++ ) { let temp = [ ] ; for ( let i = 0 ; i < N - 1 ; i ++ ) { let temp = arr [ i ] ^ 

Training: 100%|██████████| 256/256 [01:43<00:00,  2.47it/s]



BLEU: 0.0877
Pred: function findMaximumNum ( s2 ) { var setBits = ( ( 3.14 * ( ) * 100 ) / 100 ; document . write ( Math . round ( Math . round ( Math . round ( Math . round ( Math . round ( Math . round ( Math . round ( ) . toFixed ( 2 ) ) ) ; } var string = 15 ; var ( ang ) . toFixed ( 4 ) ;
Ref : function Conversion ( centi ) { let pixels = ( 96 * centi ) / 2.54 ; document . write ( pixels ) ; return 0 ; } let centi = 15 ; Conversion ( centi )

BLEU: 0.7310
Pred: function bitCount ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let i = 0 ; i < M ; i ++ ) { let temp = arr [ i ] ^ arr [ i + 1 ] ; temp . push ( value ) ; } let ans = arr [ K ] ; return ans ; } let N = 5 ; let arr = [ 1 , 4 , 5 , 6 , 7 ] ; let M = 1 ; let K = 2 ; document . write ( minDifferenceAmongMaxMin ( N , arr , M , K ) ) ;
Ref : function xor_operations ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let p = 0 ; p <

Training: 100%|██████████| 256/256 [01:41<00:00,  2.52it/s]



BLEU: 0.1140
Pred: function min ( a , b ) { return ( a * b ) / 100.0 ; } function countSetBits ( s2 ) { var setBits = ( ( 3.14 * ( ) * ( ) / ( ) ) ; document . write ( " " + 2 ) ; } var head = 15 ; var ( ) ;
Ref : function Conversion ( centi ) { let pixels = ( 96 * centi ) / 2.54 ; document . write ( pixels ) ; return 0 ; } let centi = 15 ; Conversion ( centi )

BLEU: 0.6802
Pred: function countSetBits ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( var i = 0 ; i < M ; i ++ ) { var temp = [ ] ; for ( i = 0 ; i < N - 1 ; i ++ ) { var temp = arr [ i ] ^ arr [ i + 1 ] ; temp . push ( value ) ; } arr . reverse ( ) ; return ans ; } var N = 5 ; var arr = [ 1 , 4 , 5 , 6 , 7 ] ; var M = 1 ; var K = 2 ; document . write ( ( ( ( ( ( ( ( ( ( ( ( ( N , arr , M , K ) ) ;
Ref : function xor_operations ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let p = 0 ; p < M ; p ++ ) { let temp = [ ] ; for (

In [9]:
rnn_model = AttnSeq2Seq(VOCAB_SIZE, 256, 512, pad_idx=tokenizer.pad_token_id).to(DEVICE)
rnn_model = torch.compile(rnn_model)
rnn_model.load_state_dict(torch.load("best_model.pt"))

bleu = evaluate_bleu(rnn_model, val_pairs, tokenizer, DEVICE)


BLEU: 0.1687
Pred: function perimeter ( ) { let setBits = ( ( 3.14 * hypotenuse ) / bowls ) ; document . write ( " " + " " + 2 + " " ) ; } let = " " ; let ( ) ;
Ref : function Conversion ( centi ) { let pixels = ( 96 * centi ) / 2.54 ; document . write ( pixels ) ; return 0 ; } let centi = 15 ; Conversion ( centi )


/tmp/ipykernel_109529/3309552307.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():



BLEU: 0.7234
Pred: function printCoins ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let i = 0 ; i < M ; i ++ ) { let temp = arr [ i ] ^ arr [ i + 1 ] ; temp . push ( value ) ; } let arr = new Array ( N ) ; for ( let i = 0 ; i < N ; i ++ ) { ans = arr [ i ] ^ arr [ i + 1 ] ; temp . push ( value ) ; } let ans = arr [ K ] ; return ans ; } let N = 5 ; let arr = [ 1 , 4 , 5 , 6 , 7 ] ; let M = 1 ; let K = 2 ; document . write ( " ( N + arr , M , K ) + " " ) ;
Ref : function xor_operations ( N , arr , M , K ) { if ( M < 0 M >= N ) return - 1 ; if ( K < 0 K >= N - M ) return - 1 ; for ( let p = 0 ; p < M ; p ++ ) { let temp = [ ] ; for ( let i = 0 ; i < N ; i ++ ) { let value = arr [ i ] ^ arr [ i + 1 ] ; temp . push ( value ) ; arr [ i ] = temp [ i ] ; } } let ans = arr [ K ] ; return ans ; } let N = 5 ; let arr = [ 1 , 4 , 5 , 6 , 7 ] ; let M = 1 , K = 2 ; document . write ( xor_operations ( N , arr , M , K ) ) ;

BLEU: 0.7554
Pred: func

In [10]:
test_pairs = load_pairs(
    "data/generation/pair_data_tok_full/Python-Javascript/test-Python-Javascript-tok.py",
    "data/generation/pair_data_tok_full/Python-Javascript/test-Python-Javascript-tok.js"
)

test_ds = CodeTranslationDataset(test_pairs, tokenizer)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

def evaluate_loss(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_samples = 0
    with torch.no_grad(), autocast():
        for batch in dataloader:
            src = batch["input_ids"].to(device)
            trg = batch["labels"].to(device)
            lengths = batch["lengths"].to(device)
            output = model(src, trg, lengths)
            output = output.reshape(-1, output.shape[-1])
            trg_flat = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg_flat)
            batch_size = src.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size
    return total_loss / total_samples

if __name__ == "__main__":
    test_ds = CodeTranslationDataset(test_pairs, tokenizer)
    test_loader = DataLoader(test_ds, batch_size=32, num_workers=2)
    test_loss = evaluate_loss(rnn_model, test_loader, criterion, DEVICE)
    print(f"Test Loss: {test_loss:.4f}")


/tmp/ipykernel_109529/340026405.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


Test Loss: 0.6699


In [11]:
def translate(model, tokenizer, src_text, device, max_len=256):
    model.eval()
    src_ids = tokenizer.encode(src_text, max_len)
    src_tensor = torch.tensor(src_ids).unsqueeze(0).to(device)
    lengths = (src_tensor != tokenizer.pad_token_id).sum(dim=1)
    with torch.no_grad(), autocast():
        encoder_outputs, hidden = model.encoder(
            pack_padded_sequence(model.embedding(src_tensor),
                                 lengths.cpu(),
                                 batch_first=True,
                                 enforce_sorted=False)
        )
        encoder_outputs, _ = pad_packed_sequence(encoder_outputs, batch_first=True)
        token = torch.tensor([[tokenizer.sos_token_id]], device=device)
        generated = []
        for _ in range(max_len):
            embedded = model.embedding(token)
            a = model.attn(hidden[-1], encoder_outputs).unsqueeze(1)
            weighted = torch.bmm(a, encoder_outputs)
            rnn_input = torch.cat((embedded, weighted), dim=2)
            output, hidden = model.decoder(rnn_input, hidden)
            prediction = model.fc_out(torch.cat((output, weighted), dim=2)).squeeze(1)
            token = prediction.argmax(dim=1, keepdim=True)
            if token.item() == tokenizer.eos_token_id:
                break
            generated.append(token.item())
        return tokenizer.decode(generated)


In [12]:
snippet = input("Enter Python snippet: ")
pred_js = translate(rnn_model, tokenizer, snippet, DEVICE)
print(pred_js)

KeyboardInterrupt: Interrupted by user